<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Предобработка" data-toc-modified-id="Предобработка-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#LGBMClassifier" data-toc-modified-id="LGBMClassifier-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>LGBMClassifier</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Определение негативных коментариев

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо научить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

**Цель исследования:** Построить модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`.

Возможно, что хороших результатов можно добиться и с помощью TF-IDF или Word2Vec + модели из классического ML, но мне интересно будет порабоать с нейросетью, поэтому мы будем использовать `BERT` обученую на токсичных комментариях.

**План исследования:**
* Обзор данных
* Предобработка данных
    * Избавимся от лишних символов
    * Токенизируем данные
    * Используем `BERT` предобученую на токсичных комментариях
    * Получим наши признаки для обучения моделей
* Обучение моделей
* Тестирование модели

## Обзор данных

In [6]:
# base stuff
import numpy as np
import pandas as pd
import re
import torch
import transformers
from tqdm import notebook
from tqdm import tqdm
import random
from transformers import BertTokenizer, BertModel
import optuna

# sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier

# boosting
from lightgbm import LGBMClassifier
from catboost import Pool, CatBoostClassifier

In [7]:
comments = pd.read_csv('toxic_comments.csv')

In [8]:
display(comments)
comments.info()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [9]:
comments['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

In [10]:
comments.duplicated().sum()

0

### Вывод
Данные готовы, но в них дисбаланс классов. 

## Предобработка

In [11]:
comments = comments.sample(51200, random_state=13).reset_index(drop=True)

Избавимся от всех лишних символов, оставим только латиницу:

In [12]:
def clean_text(text):
    text = re.sub(r"[^a-zA-Z' ]", ' ', text).split()
    result_text = " ".join(text)
    return result_text

In [13]:
tqdm.pandas()
comments['text'] = comments['text'].progress_apply(clean_text)
comments['text']

100%|██████████| 51200/51200 [00:01<00:00, 26179.46it/s]


0                         incorrect moronic allegations of
1        As the previous article lead already used digi...
2        to character encoding issues Oops Fixed now na...
3        Yes I know that was what Mikka did And you kno...
4                             Son of a bitchSon of a bitch
                               ...                        
51195    Preceding unsigned comment added by talk contr...
51196    need help in the india china war discussion pa...
51197    Please do not add commercial links or links to...
51198    Hanibal You're a bastard Pro Assad Hanibal You...
51199    I HATE SCIENTOLOGY IT ALL FAKE NO PROOF OF ANY...
Name: text, Length: 51200, dtype: object

Можно удалить и стоп слова, но:
* Во-первых, удаление стоп слов приведет текст к нижнему регистру, а это негативно может сказаться при обучении моделей, т.к. использование верхнего регистра ЧАСТО ЯВНЫЙ МАРКЕР ТОКСИЧНОСТИ, поэтому и `модель будем испольховать чувствительную к регистру`
* Во-вторых, с удалением стоп слов мы потеряем и все сочетания с отрицательной частицей 'not', что тоже может негативно повлиять на эффективность модели.

Токенизируем данные. Укажем, чтобы токенайзер сохранял исходный регистр:

In [14]:
tokenizer = BertTokenizer.from_pretrained('unitary/toxic-bert', do_lower_case=False)

Максимальная длина для этой модели - 512, если сообщения будут длинее - они обрежутся:

In [15]:
tokenized = comments['text'].progress_apply((lambda x: tokenizer.encode(x, max_length=512, 
                                                            truncation=True, add_special_tokens=True)))

100%|██████████| 51200/51200 [01:14<00:00, 685.02it/s]


Применим `padding`, чтобы длинна текстов в токенизированном корпусе была одинаковая, иначе bert работать не будет. (Нулями заполним пустые места)

In [16]:
padded = np.array([i + [0]*(512 - len(i)) for i in tokenized.values])

Теперь укажем, что нули, которые мы добавили - пустышки, чтобы модель их игнорировала:

In [17]:
attention_mask = np.where(padded != 0, 1, 0)

Используем преобученую на токсичных данных модель `bert`, чувствительную к регистру:

In [19]:
model = BertModel.from_pretrained("unitary/toxic-bert") 

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Теперь передадим модели тензоры, чтобы получить эмбеддинги, которые мы объеденим в матрицу, которая и будет нашими признаками для обучения:

In [20]:
%%time

batch_size = 64 
embeddings = [] 
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).cuda() # закидываем тензор на GPU
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()

        with torch.no_grad():
            model.cuda()
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy()) # перевод обратно на проц, чтобы в нумпай кинуть
        del batch
        del attention_mask_batch
        del batch_embeddings

features = np.concatenate(embeddings)
target = comments['toxic']

  0%|          | 0/800 [00:00<?, ?it/s]

CPU times: user 29min 6s, sys: 3.22 s, total: 29min 9s
Wall time: 29min 21s


### Вывод
С помощью bert, мы получили необходимые признаки для обучения моделей

## Обучение моделей

Разделим выборку на обучающую, валидационную и тестовую:

In [21]:
features_train, features_test, target_train, target_test = train_test_split(features,target, test_size=0.4, random_state=13)
features_test, features_valid, target_test, target_valid = train_test_split(features_test, target_test, test_size=0.5, random_state=13)

print(features_train.shape, features_valid.shape[0], features_test.shape[0])
target_train.shape[0], target_valid.shape[0], target_test.shape[0]

(30720, 768) 10240 10240


(30720, 10240, 10240)

Для обучения будем использовать `Optuna`, которая позволит быстрее найти самые эффективные гиперпараметры.

### LogisticRegression

In [22]:
models = []
params = []
scores = []

def objective(trial):
    C = trial.suggest_int("C", 1, 30)
    class_weight = trial.suggest_categorical("class_weight", ['balanced', None])

    model = LogisticRegression(C=C, class_weight = class_weight,
                                    random_state=13, max_iter=4000
                                   )

    model.fit(features_train, target_train)

    return f1_score(target_valid, model.predict(features_valid))

study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
study.optimize(objective, n_trials=10)

models.append('LogisticRegression')
params.append(study.best_params)
scores.append(study.best_value)

[I 2022-07-28 13:24:06,764] A new study created in memory with name: LogisticRegression
[I 2022-07-28 13:24:56,944] Trial 0 finished with value: 0.843687374749499 and parameters: {'C': 3, 'class_weight': None}. Best is trial 0 with value: 0.843687374749499.
[I 2022-07-28 13:26:12,417] Trial 1 finished with value: 0.8344965104685942 and parameters: {'C': 27, 'class_weight': None}. Best is trial 0 with value: 0.843687374749499.
[I 2022-07-28 13:27:34,162] Trial 2 finished with value: 0.8344965104685942 and parameters: {'C': 26, 'class_weight': None}. Best is trial 0 with value: 0.843687374749499.
[I 2022-07-28 13:28:55,035] Trial 3 finished with value: 0.8344965104685942 and parameters: {'C': 21, 'class_weight': None}. Best is trial 0 with value: 0.843687374749499.
[I 2022-07-28 13:29:36,504] Trial 4 finished with value: 0.7693554925010134 and parameters: {'C': 1, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.843687374749499.
[I 2022-07-28 13:30:49,370] Trial 5 finished with

### LGBMClassifier

In [23]:
def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 0.1, 1)
    n_estimators = trial.suggest_int('n_estimators', 20, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_child_samples = trial.suggest_int('min_child_samples', 5, 30)
    num_leaves = trial.suggest_int ('num_leaves', 30, 90)
    
    
    model = LGBMClassifier(learning_rate=learning_rate, 
                                    n_estimators=n_estimators,
                                    max_depth=max_depth,
                                    min_child_samples=min_child_samples, verbose=-1, 
                                    num_leaves=num_leaves, random_state=13)


    model.fit(features_train, target_train)

    return f1_score(target_valid, model.predict(features_valid))

study = optuna.create_study(study_name="LGBMClassifier", direction="maximize")
study.optimize(objective, n_trials=10)

models.append('LGBMClassifier')
params.append(study.best_params)
scores.append(study.best_value)

[I 2022-07-28 13:35:22,828] A new study created in memory with name: LGBMClassifier
[I 2022-07-28 13:36:04,328] Trial 0 finished with value: 0.8400000000000001 and parameters: {'learning_rate': 0.22386464236092768, 'n_estimators': 148, 'max_depth': 8, 'min_child_samples': 8, 'num_leaves': 31}. Best is trial 0 with value: 0.8400000000000001.
[I 2022-07-28 13:37:13,277] Trial 1 finished with value: 0.838774485183325 and parameters: {'learning_rate': 0.4391481716549816, 'n_estimators': 198, 'max_depth': 18, 'min_child_samples': 20, 'num_leaves': 57}. Best is trial 0 with value: 0.8400000000000001.
[I 2022-07-28 13:37:41,674] Trial 2 finished with value: 0.838774485183325 and parameters: {'learning_rate': 0.29557216443254636, 'n_estimators': 37, 'max_depth': 19, 'min_child_samples': 9, 'num_leaves': 85}. Best is trial 0 with value: 0.8400000000000001.
[I 2022-07-28 13:38:39,953] Trial 3 finished with value: 0.8431174089068826 and parameters: {'learning_rate': 0.18044186632795087, 'n_estima

### CatBoostClassifier

In [24]:
model = CatBoostClassifier(random_state=13, verbose=1000)
model.fit(features_train, target_train)

models.append('CatBoostClassifier')
params.append(model.get_params())
scores.append(f1_score(target_valid, model.predict(features_valid)))

Learning rate set to 0.04447
0:	learn: 0.6057593	total: 334ms	remaining: 5m 34s
999:	learn: 0.0346167	total: 3m 39s	remaining: 0us


**Результаты:**

In [25]:
final_data_results = pd.DataFrame({'f1_score': scores, 'parameters': params}, index=models).sort_values(by='f1_score')

final_data_results

,f1_score,parameters
LGBMClassifier,0.843117,"{'learning_rate': 0.18044186632795087, 'n_esti..."
LogisticRegression,0.843687,"{'C': 3, 'class_weight': None}"
CatBoostClassifier,0.845113,"{'verbose': 1000, 'random_state': 13}"


### Вывод:

Лучшая модель по метрике - `CatBoostClassifier`, но она практически не отличается `LogisticRegression`, которая обучается намного быстрее. Ее и будем тестировать.

## Тестирование модели

Перед тестированием объеденим обучающую и валидационную выборку и попробуем апсемплинг, чтобы как-то повлиять на дисбаланс классов:

In [26]:
train_and_valid_features = pd.DataFrame(features_train).append(pd.DataFrame(features_valid))
train_and_valid_target = target_train.append(target_valid)

In [29]:
model = LogisticRegression(C=3, random_state=13, max_iter=4000)
model.fit(train_and_valid_features, train_and_valid_target)
prediction = model.predict(features_test)
print(f1_score(target_test, prediction))

0.8304821150855366


upsampling

In [30]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled  = shuffle(features_upsampled, target_upsampled , random_state=12345)
    
    return features_upsampled, target_upsampled 


features_upsampled, target_upsampled = upsample(train_and_valid_features.reset_index(drop=True), train_and_valid_target.reset_index(drop=True), 10)

print(features_upsampled.shape)
print(target_upsampled.shape)

(77662, 768)
(77662,)


In [31]:
model = LogisticRegression(C=13, random_state=3, max_iter=4000)
model.fit(features_upsampled, target_upsampled)
prediction = model.predict(features_test)
print(f1_score(target_test, prediction))

0.755221386800334


Апсемплинг снизил эффектиность.

Попробуем поменять распределие признаков при разделении признаков:

In [32]:
features_train, features_test, target_train, target_test = train_test_split(features,target, test_size=0.4, random_state=13, stratify=target)
features_test, features_valid, target_test, target_valid = train_test_split(features_test, target_test, test_size=0.5, random_state=13)

train_and_valid_features = pd.DataFrame(features_train).append(pd.DataFrame(features_valid))
train_and_valid_target = target_train.append(target_valid)

In [33]:
model = LogisticRegression(C=3, random_state=13, max_iter=4000)
model.fit(train_and_valid_features, train_and_valid_target)
prediction = model.predict(features_test)
print(f1_score(target_test, prediction))

0.8337569903406202


`stratify=target` немного увеличило метрику `f1`

Проверим нашу модель на адекватность с помощью `DummyClassifier`:

In [34]:
model = DummyClassifier(random_state=13)
model.fit(features_upsampled, target_upsampled)
predictions = model.predict(features_test)
print(f1_score(predictions, target_test))

0.17874610938194754


Показатели нашей модели выше - она адекватна

## Вывод


Мы смогли добиться метрики `f1 = 0.83`, c `LogisticRegression` c гиперпараметром `C=3`. 

Похоже, что `BERT` довольно тяжелый и нельзя расчитывать на его быструю работу без больших ресурсов. Возможно, стоит попробовать использовать и другие методики (TF-IDF или Word2Vec) или другие готовые модели, может они справятся не хуже и быстрее.